# 1. Type Token Ratio
- 개요
    - 유형 토큰 비율(TTR)은 언어적 다양성을 측정하는 방법으로, 고유 토큰의 비율을 총 토큰 수로 나눈 값으로 정의.
    - 측정값은 0과 1 사이로 제한되며 텍스트에 반복이 없으면 이 측정값은 1이고 무한 반복이 있으면 0에 가까워짐.
    - 토큰 수가 증가하면 TTR이 평평해지는 경향이 있으므로 길이가 다른 텍스트를 분석하는 경우에는 이 측정값을 권장하지는 않음.
- 해석
    - 텍스트나 문서의 TTR 값이 낮으면 기능어가 내용어보다 더 많다는 뜻.
        - 기능어는 대명사, 전치사, 수정 동사와 같이 문장의 내용을 둘러싸는 역할을 하는 언어의 필러 단어
    - TTR은 텍스트 복잡성 자체를 나타낼 수 없음.

In [3]:
from konlpy.tag import Okt
from huggingface_hub import hf_hub_download
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("HF_API_KEY")
repo_id = "SaeSSak/Conversation"
file_path = hf_hub_download(repo_id, "FinalData.parquet", repo_type="dataset", use_auth_token=api_key)
data = pd.read_parquet(file_path)

/home/jinhakai2/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
okt = Okt()

In [27]:
sen1 = "화장실에 갈 때도 줄을 잘 서야 해요"
sen2 = "밥 먹으러 가자"

## 1-1. 접근 1: 어근의 비율

In [33]:
def root_tts(text):
    tokens = okt.morphs(text)
    roots = [okt.morphs(word, stem=True) for word in tokens]
    flat_roots = [item for sublist in roots for item in sublist]
    print(text)
    print(tokens)
    print(flat_roots)

In [35]:
root_tts(sen1)
root_tts(sen2)

화장실에 갈 때도 줄을 잘 서야 해요
['화장실', '에', '갈', '때', '도', '줄', '을', '잘', '서야', '해', '요']
['화장실', '에', '갈다', '때', '도', '줄', '을', '자다', '서다', '해', '요']
밥 먹으러 가자
['밥', '먹으러', '가자']
['밥', '먹다', '가다']


In [26]:
okt.morphs(tokens[2], stem=True)

['갈다']

In [5]:
# TTR 계산 함수
def calculate_ttr(text):
    okt = Okt()
    tokens = okt.morphs(text)
    unique_tokens = set(tokens)
    return len(unique_tokens) / len(tokens) if tokens else 0

,음성인식결과,지역,성별,대화주제,나이
0,내일 또 얘기하자,수도권,여,학교,5
1,교복을 입고 왔어요,수도권,여,교복,5
2,나는 씩씩해요,수도권,여,소풍을 다녀와서,7
3,빨리 내일이 됐으면 좋겠다,수도권,여,삼촌댁에 놀러가기,7
4,같이 맞는 거야,수도권,여,삶과 죽음,7
...,...,...,...,...,...
1142358,나도 발레 배우는데,수도권,남,발레,10
1142359,유치원 때부터 배웠어,수도권,남,발레,10
1142360,다리를 모아야 돼,수도권,남,발레,10
1142361,무릎이 붙어야 하는 거야,수도권,남,발레,10


# 2. HD-D (Hypergeometric Distribution D)

In [48]:
import nltk
from nltk.probability import FreqDist
from scipy.stats import hypergeom
import numpy as np

# 텍스트 샘플
text = """
Hypergeometric Distribution D (HDD)는 각 단어의 텍스트 내 등장 빈도와 텍스트의 길이를 기반으로 어휘 다양성을 계산하는 방법입니다. HDD는 특히 다양한 길이의 텍스트에서 일관된 측정을 제공합니다.

HDD를 계산하는 기본 원리는 다음과 같습니다: 각 단어에 대해, 만약 단어가 텍스트에서 무작위로 선택된 n개의 단어 중 하나라면, 그 단어가 적어도 한 번 나타날 확률을 계산합니다. 이러한 확률들의 합은 전체 텍스트의 어휘 다양성을 나타냅니다.

Python으로 HDD를 구현하는 예제를 제공하겠습니다. 이 예제에서는 nltk 라이브러리를 사용하여 텍스트를 토큰화하고 빈도를 계산합니다. scipy 라이브러리의 hypergeom 모듈을 사용하여 필요한 확률을 계산합니다.
"""

# 토큰화 및 빈도 분석
tokens = nltk.word_tokenize(text)
fdist = FreqDist(tokens)

In [49]:
N = len(tokens)
n = N // 10  # 예를 들어, 텍스트 길이의 약 10%를 샘플 크기로 설정
hdd = sum(hypergeom.sf(0, N, fdist[word], n) for word in fdist)

print(f"HDD 값: {hdd}")

HDD 값: 8.732897548968863


# 2. MTLD (Measure of Textual Lexical Diversity)

In [ ]:
import nltk
from nltk.probability import FreqDist
from scipy.stats import hypergeom
import numpy as np

# 텍스트 샘플
text = "화장실에 갈 때도 줄을 잘 서야 해요. 화장실을 사용할 때는 질서를 지키는 것이 중요합니다."

# 토큰화 및 빈도 분석
tokens = nltk.word_tokenize(text)